In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import math

class node:
    def __init__(self, featureIndex=None,rightC=None,leftC=None, gain=None,data=None,thresh=None):
        self.rightC=rightC
        self.leftC=leftC
        self.data=data
        self.featureIndex=featureIndex
        self.gain=gain
        self.thresh=thresh
    def printTree(self, level=0):
        if self.data:
            print("depth: ",level,"index ", self.featureIndex, "gain: ", self.gain, "data: ", self.data, "thresh: ",self.thresh)
            if self.leftC:
                self.leftC.printTree( level+1, )
            if self.rightC:
                self.rightC.printTree(level+1)
    def find(self, feature):
        if self.data!=-1:
            return self.data
        else:
            if feature[self.featureIndex]<self.thresh:
                return self.leftC.find(feature)
            else:
                return self.rightC.find(feature)


In [187]:
class builderTree:
    def __init__(self,depthLimit=float("inf")):
        self.depthLimit=depthLimit
        self.root=None
    def fit(self, featureX, classY):
        self.root=self.buildTree(featureX,classY)
    def buildTree(self,featureX,classY,depth=0):
        infoGain=-1
        thresh=-1
        index=-1
        if len(classY) == 0:
            return None
        elif len(classY)== 1:
            return node(None, None, None,0,classY[0])
        elif np.all(classY[0]==classY[:]):
            return node(None, None, None,0,classY[0])
        else:
            for colItr in range(len(featureX[1])):
#                 allvalues = np.array(featureX[:, colItr])
                allvalues=[]
                for d in range(len(featureX)):
                    allvalues.append(featureX[d][colItr])
                colMean = np.mean(allvalues)
                xLeft, yLeft, xRight, yRight= makePartition(featureX,classY,colItr,colMean)
                infoGain2=infoGainCalc(classY,yLeft,yRight)
                if infoGain<infoGain2:
                    infoGain=infoGain2
                    index=colItr
                    thresh=colMean
            xLeft, yLeft, xRight, yRight= makePartition(featureX, classY, index,thresh)
            depth+=1
            leftChild=self.buildTree(xLeft,yLeft,depth)
            rightChild=self.buildTree(xRight,yRight,depth)
            return node(index,rightChild,leftChild,infoGain,-1,thresh)
                
def makePartition(featureX, classY, compareIdx, colMean):
    retLeftX=[]
    retLeftY=[]
    retRightX=[]
    retRightY=[]

    for itr in range(len(featureX)):
        if featureX[itr][compareIdx]<colMean:
            retLeftX.append(featureX[itr])
            retLeftY.append(classY[itr])
        else:
            retRightX.append(featureX[itr])
            retRightY.append(classY[itr])
    return retLeftX, retLeftY, retRightX, retRightY

def infoGainCalc(classY, yLeft, yRight):
    infoPar=entropy(classY)
    parSize=len(classY)
    lSize=len(yLeft)
    rSize=len(yRight)
    e1=entropy(yLeft)
    e2=entropy(yRight)
#     print(e1)
#     print(e2)
#     print(lSize)
#     print(rSize)
    infoChild=lSize*e1+ rSize*e2
    infoChild = infoChild/parSize;
    gain=infoPar-infoChild
    return gain
    
def entropy(theList):
    everything= len(theList)
    uniqueList= np.unique(theList)
    entropy=0
    for itr in uniqueList:
        curr=np.count_nonzero(theList == itr)
        probi= curr/everything
        entropy=entropy + probi * math.log2(1/probi)
    return entropy
X_train=None
X_test=None
Y_train=None
Y_test=None
# reading csv files
names= ["class","Alcohol", "Malic acid"
,"Ash",
 "Alcalinity of ash",  
"Magnesium",
"Total phenols",
"Flavanoids",
"Nonflavanoid phenols",
  "Proanthocyanins",
"Color intensity",
 "Hue",
 "OD280/OD315 of diluted wines",
"Proline" ]
data =  pd.read_csv('wine.data', sep=",")
# data.columns=names
X= data.iloc[:,1:15];
Y=data.iloc[:,0:1];
X_train, X_test, Y_train, Y_test= train_test_split(X,Y,test_size=0.1,random_state=33)
X_train=np.array(X_train)
Y_train=np.array(Y_train)
X_test=np.array(X_test)
Y_test=np.array(Y_test)
hoileBhalo=builderTree()
hoileBhalo.fit(X_train, Y_train)
# print(X_train[1])
# for i in range(len(X_train[0])):
#     print(i)
count=0;
total=len(X_test)
for i in range(len(X_test)):
    toTest=X_test[i,:]
    if Y_test[i]==hoileBhalo.root.find(toTest)[0]:
        count=count+1
print("Accuracy: ", count/total)
hoileBhalo.root.printTree()

Accuracy:  0.9444444444444444
depth:  0 index  6 gain:  0.5889523439093397 data:  -1 thresh:  2.0057232704402512
depth:  1 index  10 gain:  0.5085074010161712 data:  -1 thresh:  0.8466133333333332
depth:  2 index  6 gain:  0.11358850854373598 data:  -1 thresh:  0.8670000000000002
depth:  3 index  None gain:  0 data:  [3] thresh:  None
depth:  3 index  7 gain:  0.3219280948873623 data:  -1 thresh:  0.3486666666666667
depth:  4 index  None gain:  0 data:  [3] thresh:  None
depth:  4 index  4 gain:  1.0 data:  -1 thresh:  86.66666666666667
depth:  5 index  None gain:  0 data:  [2] thresh:  None
depth:  5 index  None gain:  0 data:  [3] thresh:  None
depth:  2 index  1 gain:  0.27926328354671254 data:  -1 thresh:  1.88
depth:  3 index  None gain:  0 data:  [2] thresh:  None
depth:  3 index  0 gain:  0.9940302114769566 data:  -1 thresh:  12.535454545454543
depth:  4 index  None gain:  0 data:  [2] thresh:  None
depth:  4 index  None gain:  0 data:  [3] thresh:  None
depth:  1 index  0 gain:

0.0